<a href="https://colab.research.google.com/github/LanceTBag/Data-Clean-and-Split---November/blob/main/Revised_split_and_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract(input_file, separator):
    try:
        # Read the CSV file with the user-specified separator
        df = pd.read_csv(input_file, encoding='utf-8', sep=separator)
        logging.info(f"Successfully loaded {input_file} with separator '{separator}'")
        return df
    except Exception as e:
        logging.error(f"Error reading {input_file}: {e}")
        raise e

def transform(df, input_file):
    # Initialize invalid rows dataframe and issue column
    invalid_rows = pd.DataFrame(columns=df.columns.tolist() + ['Issue'])

    # Clean invalid characters
    def clean_text(text):
        if isinstance(text, str):
            return re.sub(r'â€|[^\x00-\x7F]+', '', text)  # Removes non-ASCII characters
        return text

    # Apply cleaning functions
    df_clean = df.copy()

    # Step 1: Clean text for all columns
    for column in df_clean.columns:
        df_clean[column] = df_clean[column].map(clean_text)

    # Step 2: Handle missing values (remove records where login_id or mail_address is missing)
    missing_login_id_rows = df_clean[df_clean['login_id'].isnull()]
    if not missing_login_id_rows.empty:
        missing_login_id_rows['Issue'] = 'Missing login_id'
        invalid_rows = pd.concat([invalid_rows, missing_login_id_rows], ignore_index=True)

    missing_mail_address_rows = df_clean[df_clean['mail_address'].isnull()]
    if not missing_mail_address_rows.empty:
        missing_mail_address_rows['Issue'] = 'Missing mail_address'
        invalid_rows = pd.concat([invalid_rows, missing_mail_address_rows], ignore_index=True)

    df_clean = df_clean.dropna(subset=['login_id', 'mail_address'])

    # Step 3: Handle duplicates by keeping one and removing the others
    duplicate_rows = df_clean[df_clean.duplicated(subset=['login_id', 'mail_address'], keep='first')]
    if not duplicate_rows.empty:
        duplicate_rows['Issue'] = 'Duplicate'
        invalid_rows = pd.concat([invalid_rows, duplicate_rows], ignore_index=True)
    df_clean = df_clean.drop_duplicates(subset=['login_id', 'mail_address'], keep='first')

    # Update the garbage file name based on input file
    base_filename = os.path.splitext(os.path.basename(input_file))[0]
    garbage_file = f"{base_filename}_garbage.csv"

    return df_clean, invalid_rows, garbage_file

def load(df_cleaned, output_file, invalid_rows, garbage_file):
    try:
        # Save the cleaned DataFrame to a new CSV file
        df_cleaned.to_csv(output_file, index=False, encoding='utf-8', sep=',')
        logging.info(f"Cleaned data written to {output_file}")

        # Save the invalid/garbage records to a separate CSV file
        invalid_rows.to_csv(garbage_file, index=False, encoding='utf-8', sep=',')
        logging.info(f"Invalid records written to {garbage_file}")
    except Exception as e:
        logging.error(f"Error writing to file: {e}")
        raise e

def split_file(output_file, chunks):
    # Read the cleaned output file
    df = pd.read_csv(output_file, encoding='utf-8')
    rows_per_chunk = len(df) // chunks
    base_filename = os.path.splitext(output_file)[0]

    # Split the file into chunks
    for i in range(chunks):
        start = i * rows_per_chunk
        end = None if i == chunks - 1 else (i + 1) * rows_per_chunk
        chunk = df.iloc[start:end]
        split_file_name = f"{base_filename}_{i + 1}.csv"
        chunk.to_csv(split_file_name, index=False, encoding='utf-8', sep=',')
        logging.info(f"Split file {split_file_name} created with {len(chunk)} records")

def modify_created_at(df):
    if 'created_at' in df.columns:
        # Modify the created_at column to only keep the 'yyyy-mm-dd' format
        df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce').dt.date
    return df

def main():
    # Prompt for input file, separator, and chunk number
    input_file = input("Enter the input CSV file name: ")
    separator = input("Enter the separator used in the input CSV file: ")
    chunks = int(input("Enter the number of chunks to split the file into: "))

    # Generate filenames based on the input file
    base_filename = os.path.splitext(os.path.basename(input_file))[0]
    output_file = f"{base_filename}_nonduplicate.csv"

    # Step 1: Extract
    df = extract(input_file, separator)

    # Step 2: Transform
    df_cleaned, invalid_rows, garbage_file = transform(df, input_file)

    # Modify 'created_at' column in the cleaned DataFrame
    df_cleaned = modify_created_at(df_cleaned)

    # Step 3: Load cleaned and invalid data to respective files
    load(df_cleaned, output_file, invalid_rows, garbage_file)

    # Step 4: Split cleaned file into chunks
    split_file(output_file, chunks)

if __name__ == "__main__":
    main()


Enter the input CSV file name: /content/Japan-2019.csv
Enter the separator used in the input CSV file: ;
Enter the number of chunks to split the file into: 7


<ipython-input-2-e090c16dbd39>:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, encoding='utf-8', sep=separator)
<ipython-input-2-e090c16dbd39>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_login_id_rows['Issue'] = 'Missing login_id'
<ipython-input-2-e090c16dbd39>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  invalid_rows = pd.concat([invalid_rows, missing_login_id_rows], ignore_index=True)
<ipython-i